# Lab 11: Trees

## <font color=DarkRed>Your Exercise: Parsing Expressions with Boolean Operators</font>

Modify the `build_parse_tree`, and `evaluate` functions to handle boolean operators (`and`, `or`, and `not`). Remember that `not` is a unary operator, so this will complicate your code somewhat.

## <font color=green>Your Solution</font>

*Use a variety of code, Markdown (text) cells below to create your solution. Nice outputs would be timing results, and even plots. You will be graded not only on correctness, but the clarity of your code, descriptive text and other output. Keep it succinct!*

In [23]:
import string
import operator

In [24]:
class Stack():
    def __init__(self):
        self.items = []
    
    def is_empty(self):
        return self.items == []

    def size(self):
        return len(self.items)

    def peek(self):
        return self.items[-1]
  
    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

In [25]:
class BinaryTree:
    def __init__(self, key):
        self.key = key
        self.left_child = None
        self.right_child = None
        
    def insert_left(self, key):
        if self.left_child is None:
            self.left_child = BinaryTree(key)
        else:  # if there IS a left child
            t = BinaryTree(key)
            t.left_child = self.left_child
            self.left_child = t
            
    def insert_right(self, key):
        if self.right_child is None:
            self.right_child = BinaryTree(key)
        else:  # if there IS a right child
            t = BinaryTree(key)
            t.right_child = self.right_child
            self.right_child = t
            
    def get_right_child(self):
        return self.right_child
    
    def get_left_child(self):
        return self.left_child
    
    def get_root_val(self):
        return self.key
    
    def set_root_val(self, new_key):
        self.key = new_key
    
    def __repr__(self):
        return f"BinaryTree({self.key!r})"

In [26]:
def build_parse_tree(fpexp):
    fplist = fpexp.split()
    p_stack = Stack()
    e_tree = BinaryTree('')
    
    p_stack.push(e_tree)
    current_tree = e_tree
    
    for i in fplist:
        if i == "(":
            current_tree.insert_left('')
            p_stack.push(current_tree)
            current_tree = current_tree.get_left_child()
        elif i.isdigit():
            current_tree.set_root_val(int(i))
            parent = p_stack.pop()
            current_tree = parent
        elif i in ["+", "-", "/", "*", "and", "or"]:
            current_tree.set_root_val(i)
            current_tree.insert_right('')
            p_stack.push(current_tree)
            current_tree = current_tree.get_right_child()
        elif i in ["not"]:
            current_tree.set_root_val(None)  #set left node to None
            current_tree = p_stack.pop()     #go back to parent
            current_tree.set_root_val(i)
            p_stack.push(current_tree)
            current_tree = current_tree.get_left_child()          
        elif i == ")":
            current_tree = p_stack.pop()
        else:
            raise ValueError("invalid expression given!")          
    return e_tree

In [27]:
def evaluate(parse_tree):
    opers = {
        "+": operator.add,
        "-": operator.sub,
        "*": operator.mul,
        "/": operator.truediv,
        "and": operator.and_,
        "or": operator.or_,
        "not": operator.not_,
    }
    
    left_c = parse_tree.get_left_child()
    right_c = parse_tree.get_right_child()
    
    if left_c and right_c:
        fn = opers[parse_tree.get_root_val()]
        return fn(evaluate(left_c), evaluate(right_c))
    elif left_c:
        fn = opers[parse_tree.get_root_val()]
        return fn(evaluate(left_c))
    else:
        return parse_tree.get_root_val()

## Testing

Test `build_parse_tree`, and `evaluate` to show that boolean expressions (with, or without arithmetic expressions mixed), work as expected.

In [28]:
print(operator.and_(0, 888))
print(operator.and_(233, 888))

0
104


In [29]:
pt1 = build_parse_tree(" ( 0 and 888 )")
pt2 = build_parse_tree(" ( 233 and 888 )")
print(evaluate(pt1))
print(evaluate(pt2))

0
104


In [30]:
print(operator.or_(0, 0))
print(operator.or_(0, 888))
print(operator.or_(233, 888))

0
888
1017


In [31]:
pt3 = build_parse_tree(" ( 0 or 0 )")
pt4 = build_parse_tree(" ( 0 or 888 )")
pt5 = build_parse_tree(" ( 233 or 888 )")
print(evaluate(pt3))
print(evaluate(pt4))
print(evaluate(pt5))

0
888
1017


In [32]:
print(operator.not_(0))
print(operator.not_(888))

True
False


In [33]:
pt6 = build_parse_tree(" ( not 0 )")
pt7 = build_parse_tree(" ( not 888 )")
print(evaluate(pt6))
print(evaluate(pt7))

True
False
